In [7]:
import pandas as pd

df = pd.read_csv('Argument Appropriateness - appropriateness_corpus_conservative_w_folds.csv')
df = df[['issue','post_text','Inappropriateness','fold0.0']]

selected_df = df[['post_text','Inappropriateness', 'fold0.0']]
selected_df = selected_df.rename(columns={'post_text': 'txt', 'Inappropriateness': 'style', 'fold0.0': 'split'})

#replace 1 with inapp and 0 with app in style column
selected_df['style'] = selected_df['style'].replace([1, 0], ['inapp', 'app'])

#rename TEST, TRAIN, VALID to test, train, val
selected_df['split'] = selected_df['split'].replace(['TEST', 'TRAIN', 'VALID'], ['test', 'train', 'val'])

#reorder
selected_df = selected_df[['txt','split', 'style']]

#convert to tsv
selected_df.to_csv('tag-and-generate-data-prep/data/inapp/Inappropriateness.tsv', sep='\t', index=False, header=True)

In [30]:
!python3 tag-and-generate-data-prep/src/run.py --data_pth tag-and-generate-data-prep/data/inapp/Inappropriateness.tsv --outpath tag-and-generate-data-prep/data/tmp --style_0_label 'inapp' --style_1_label 'app' --is_unimodal False

/home/shady/Environments/miniconda3/envs/taggen/lib/python3.7/site-packages/tqdm/_tqdm.py:634: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
INFO:root:Reading the data
INFO:root:Getting TF-IDF stats for both the corpora
INFO:root:#Records inapp = 1182
INFO:root:#Records app = 1009
INFO:root:Running TfidfVectorizer
INFO:root:Running TfidfVectorizer
/home/shady/Projects/tagger-generator/tag-and-generate-data-prep/src/style_tags.py:266: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  important_phrases["score"] = (important_phrases["rank"] - self.thresh) / (1 - self.thresh)
INFO:root:Generating the tagged data
100%|█████████████████████

In [36]:
!bash tag-and-generate-train/scripts/prepare_bpe.sh tagged tag-and-generate-data-prep/data/tmp
!bash tag-and-generate-train/scripts/prepare_bpe.sh generated tag-and-generate-data-prep/data/tmp

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --model_prefix=tag-and-generate-data-prep/data/tmp/entagged_subwords --vocab_size=16000 --model_type=bpe --input=tag-and-generate-data-prep/data/tmp/entagged_parallel.train.tagged,tag-and-generate-data-prep/data/tmp/entagged_parallel.train.en
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: tag-and-generate-data-prep/data/tmp/entagged_parallel.train.tagged
  input: tag-and-generate-data-prep/data/tmp/entagged_parallel.train.en
  input_format: 
  model_prefix: tag-and-generate-data-prep/data/tmp/entagged_subwords
  model_type: BPE
  vocab_size: 16000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  sp

In [2]:
!bash tag-and-generate-train/scripts/train_tagger.sh tagged appropriatness tag-and-generate-data-prep/data/tmp 
!bash tag-and-generate-train/scripts/train_generator.sh generated appropriatness tag-and-generate-data-prep/data/tmp 

Using BPE
loading tag-and-generate-data-prep/data/tmp/entagged_parallel.bpe.train tag-and-generate-data-prep/data/tmp/entagged_parallel.bpe.dev
----- Epoch 1 -----
0it [00:00, ?it/s]Train with noisy input :  False
Train with noisy output.  False
100%|███████████████████| 58/58 [01:04<00:00,  1.11s/it, loss=6.643, ppl=767.67]
10it [00:03,  3.06it/s]
Validation perplexity: 896.11
Saving new best model (epoch 1 ppl 896.109955766201)
----- Epoch 2 -----
  0%|                                                    | 0/58 [00:00<?, ?it/s]Train with noisy input :  False
Train with noisy output.  False
100%|███████████████████████████████████████████| 10/10 [00:03<00:00,  3.09it/s]
Validation perplexity: 620.23
Saving new best model (epoch 2 ppl 620.2313662457155)
----- Epoch 3 -----
  0%|                                                    | 0/58 [00:00<?, ?it/s]Train with noisy input :  False
Train with noisy output.  False
100%|███████████████████████████████████████████| 10/10 [00:03<00:00,  3.